In [1]:
import numpy as np
import simpy
import random

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
SEED = 930
UNIFORM_INTERVAL = [4.17, 5.37]
NUM_SERVER = 2
SERVER_UTILIZATIONS_PHASE1 = [0.6, 0.7, 0.8, 0.9]
SERVER_SERVICE_MEAN_PHASE1 = [5.725,6.679,7.634,8.588]
SERVER_UTILIATION_PHASE2 = 0.8
NUMBER_OF_CUSTOMERS = 100


In [5]:
phase_1 = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIZATIONS_PHASE1,"serv_mean":SERVER_SERVICE_MEAN_PHASE1}
phase_2 = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIATION_PHASE2,"serv_mean":7.634}

In [7]:
class Jobs(object):
    def __init__(self, name, env, operator , run_):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.run = run_
    
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            self.run.queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.ask_question())
            print('%s is done at %g' % (self.name, self.env.now))
            
    def ask_question(self):
        duration = random.expovariate(self.phase.serv_mean)
        yield self.env.timeout(duration)
        self.run.service_times.append(duration)
        
        

In [8]:
class Run():
    def __init__(self,arr_rate,serv_rate):
        self.seed = random.random()*425
        
        self.arr_rate = arr_rate
        self.serv_rate = serv_rate
        self.jobs=[]
        self.service_times=[]
        self.queue_w_times=[]
        self.env = simpy.Environment()
        self.operator = simpy.Resource(self.env, capacity = 1)
    
    def run_sim(self):
        random.seed(self.seed)
        self.env.process(self.jobs_generator())
        self.env.run()
        pass


    def jobs_generator(self):
        for i in range(NUMBER_OF_CUSTOMERS):
            yield self.env.timeout(random.expovariate(self.arr_rate))
            job = Jobs('Cust %s' %(i+1), self.env, self.operator,self)
            self.jobs.append(job)  

In [9]:
class Phase():
    def __init__(self,lib,num_runs):
        self.num_server = lib["num_server"]
        self.arr_int = lib["arr_interval"]
        self.util = lib["util"]
        self.serv_mean = lib["serv_mean"]
        self.init_runs(num_runs)


    def init_runs(self,num_runs):
        self.runs = []
        for util_no in range(len(self.util)):
            for i in range(num_runs):
                obj = Run(self.arr_int,self.serv_mean[util_no])
                self.runs.append(obj)

    def run_phase(self):
        for run in self.runs:
            run.run_sim()
    
